# Integración CoreML de RusTorch - Kernel Rust

Este notebook demuestra cómo usar CoreML con RusTorch.
Se ejecuta en el kernel Rust.

## Verificar Dependencias y Características Requeridas

In [ ]:
// Uso básico de RusTorch
extern crate rustorch;

use rustorch::tensor::Tensor;
use rustorch::gpu::DeviceType;

println!("Versión de RusTorch: {}", env!("CARGO_PKG_VERSION"));
println!("Versión de Rust: {}", env!("RUSTC_VERSION"));

## Verificar Disponibilidad de CoreML

Verificar si CoreML está disponible en el sistema actual.

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::backends::DeviceManager;
    
    let coreml_available = DeviceManager::is_coreml_available();
    println!("CoreML disponible: {}", coreml_available);
    
    if coreml_available {
        println!("🎉 ¡CoreML está disponible!");
        println!("Plataforma: macOS");
        
        // Mostrar información del dispositivo
        use rustorch::gpu::coreml::device_cache::DeviceCache;
        let cache = DeviceCache::global();
        cache.warmup();
        
        let stats = cache.get_stats();
        println!("Estadísticas de caché: {:?}", stats);
    } else {
        println!("⚠️ CoreML no está disponible");
        println!("Por favor use CPU u otros backends GPU");
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("❌ Las características CoreML no están habilitadas");
    println!("Por favor construya con --features coreml");
}

## Operaciones Básicas de Tensores

In [ ]:
// Crear tensores básicos
let a = Tensor::zeros(&[2, 3]);
let b = Tensor::ones(&[3, 2]);

println!("Forma del tensor A: {:?}", a.shape());
println!("Forma del tensor B: {:?}", b.shape());

// Multiplicación de matrices básica
let result = a.matmul(&b);
println!("Forma del resultado: {:?}", result.shape());
println!("Operaciones básicas de tensores completadas");

## Operaciones con Dispositivo CoreML

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::gpu::coreml::{CoreMLDevice, CoreMLBackend};
    use rustorch::backends::BackendConfig;
    
    // Intentar crear dispositivo CoreML
    match CoreMLDevice::new(0) {
        Ok(device) => {
            println!("🖥️ Dispositivo CoreML creado exitosamente");
            println!("ID del dispositivo: {}", device.id());
            println!("Disponible: {}", device.is_available());
            println!("Límite de memoria: {} MB", device.memory_limit() / (1024 * 1024));
            
            // Crear configuración del backend
            let config = BackendConfig::new()
                .with_caching(true)
                .with_max_cache_size(200)
                .with_profiling(true)
                .with_auto_fallback(true);
            
            println!("⚙️ Configuración del backend: {:?}", config);
            
            // Crear backend CoreML
            match CoreMLBackend::new(device, config) {
                Ok(backend) => {
                    println!("🚀 Backend CoreML inicializado");
                    
                    // Obtener estadísticas
                    let stats = backend.get_statistics();
                    println!("📊 Estadísticas del backend:");
                    println!("  Operaciones totales: {}", stats.total_operations);
                    println!("  Aciertos de caché: {}", stats.cache_hits);
                    println!("  Fallos de caché: {}", stats.cache_misses);
                    println!("  Operaciones de respaldo: {}", stats.fallback_operations);
                    
                    // Crear tensores en CoreML
                    let tensor_a = Tensor::randn(&[64, 64]).to_device(&backend);
                    let tensor_b = Tensor::randn(&[64, 64]).to_device(&backend);
                    
                    println!("📐 Tensores creados en dispositivo CoreML");
                    
                    // Operación de multiplicación de matrices
                    let start = std::time::Instant::now();
                    let result = tensor_a.matmul(&tensor_b);
                    let duration = start.elapsed();
                    
                    println!("✅ Multiplicación de matrices completada");
                    println!("⏱️ Tiempo de ejecución: {:?}", duration);
                    println!("🎯 Forma del resultado: {:?}", result.shape());
                    
                    // Limpiar caché
                    backend.cleanup_cache();
                    println!("🧹 Caché limpiado");
                }
                Err(e) => println!("❌ Error al crear backend CoreML: {:?}", e),
            }
        }
        Err(e) => {
            println!("❌ Error al crear dispositivo CoreML: {:?}", e);
            println!("Puede que CoreML no esté disponible en este sistema");
        }
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ Omitiendo operaciones CoreML - características no habilitadas");
}

## Comparación de Rendimiento

In [ ]:
use std::time::Instant;

fn benchmark_operations() {
    let sizes = vec![(64, 64), (128, 128), (256, 256), (512, 512)];
    
    println!("🏁 Evaluación comparativa de operaciones:");
    println!("Tamaño\t\tCPU (ms)\tDispositivo Preferido");
    println!("-" * 45);
    
    for (rows, cols) in sizes {
        // Crear tensores en CPU
        let a = Tensor::randn(&[rows, cols]);
        let b = Tensor::randn(&[cols, rows]);
        
        // Medir tiempo CPU
        let start = Instant::now();
        let _result = a.matmul(&b);
        let cpu_duration = start.elapsed();
        
        // Determinar dispositivo preferido
        let preferred_device = if rows * cols < 1000 {
            "CPU"
        } else if rows * cols < 10000 {
            "Metal GPU"
        } else {
            "CoreML"
        };
        
        println!("{}x{}\t\t{:.2}\t\t{}", 
                rows, cols, 
                cpu_duration.as_millis() as f64, 
                preferred_device);
    }
}

benchmark_operations();
println!("\n📝 Nota: La selección de dispositivo se basa en tamaño de tensor y disponibilidad");

## Selección Inteligente de Dispositivo

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::backends::{DeviceManager, DeviceSelector};
    
    fn demonstrate_device_selection() {
        println!("🎯 Demostración de selección inteligente de dispositivo:");
        
        let operations = vec![
            ("Multiplicación pequeña", vec![16, 16], "CPU"),
            ("Convolución 2D", vec![32, 3, 224, 224], "CoreML"),
            ("Transformación de matriz", vec![128, 128], "Metal GPU"),
            ("Operación de lote grande", vec![512, 512], "CoreML"),
            ("Cálculo vectorial", vec![1000], "CPU"),
        ];
        
        for (name, shape, preferred) in operations {
            println!("  {:<25} {:?} -> {}", name, shape, preferred);
            
            // Simular selección basada en reglas
            let tensor_size: usize = shape.iter().product();
            let selected_device = match tensor_size {
                size if size < 1000 => DeviceType::Cpu,
                size if size < 50000 => DeviceType::MetalGpu,
                _ => {
                    if DeviceManager::is_coreml_available() {
                        DeviceType::CoreML
                    } else {
                        DeviceType::MetalGpu
                    }
                }
            };
            
            println!("    -> Dispositivo seleccionado: {:?}", selected_device);
        }
        
        println!("\n📝 Lógica de selección:");
        println!("  • < 1K elementos: CPU (sobrecarga mínima)");
        println!("  • 1K-50K elementos: Metal GPU (equilibrado)");
        println!("  • > 50K elementos: CoreML (optimizado) o Metal GPU (respaldo)");
    }
    
    demonstrate_device_selection();
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ Demostración de selección de dispositivo omitida - características CoreML no disponibles");
}

## Ejemplo Avanzado: Capa de Red Neuronal

In [ ]:
fn simulate_neural_layer() {
    println!("🧠 Simulación de capa de red neuronal:");
    
    // Configuración de la capa
    let batch_size = 32;
    let input_dim = 784;   // 28x28 MNIST
    let hidden_dim = 256;
    let output_dim = 10;   // 10 clases
    
    println!("📊 Configuración:");
    println!("  Tamaño del lote: {}", batch_size);
    println!("  Dimensión de entrada: {}", input_dim);
    println!("  Dimensión oculta: {}", hidden_dim);
    println!("  Dimensión de salida: {}", output_dim);
    
    // Crear tensores
    let input = Tensor::randn(&[batch_size, input_dim]);
    let weight1 = Tensor::randn(&[input_dim, hidden_dim]);
    let weight2 = Tensor::randn(&[hidden_dim, output_dim]);
    
    println!("\n🔄 Pase hacia adelante:");
    
    // Pase hacia adelante simulado
    let start = Instant::now();
    
    // Capa 1: entrada -> oculta
    let hidden = input.matmul(&weight1);
    println!("  ✅ Entrada -> Oculta: {:?}", hidden.shape());
    
    // Función de activación ReLU (simulada)
    let activated = hidden.relu();
    println!("  ✅ Activación ReLU aplicada");
    
    // Capa 2: oculta -> salida
    let output = activated.matmul(&weight2);
    println!("  ✅ Oculta -> Salida: {:?}", output.shape());
    
    let total_time = start.elapsed();
    
    println!("\n⏱️ Tiempo total del pase hacia adelante: {:?}", total_time);
    println!("🚀 Rendimiento estimado: {:.0} muestras/segundo", 
             (batch_size as f64) / total_time.as_secs_f64());
    
    println!("\n📝 En una implementación real:");
    println!("  • Matrices grandes usarían CoreML");
    println!("  • Activaciones usarían Metal GPU");
    println!("  • Operaciones pequeñas permanecerían en CPU");
}

simulate_neural_layer();

## Manejo de Errores y Respaldo

In [ ]:
fn demonstrate_fallback_behavior() {
    println!("🔄 Demostración de comportamiento de respaldo:");
    
    // Simular operación que podría fallar en CoreML
    let complex_tensor = Tensor::randn(&[100, 100]);
    
    println!("🎯 Intentando operación CoreML...");
    
    // En implementación real, esto sería:
    // match tensor.to_coreml() {
    //     Ok(coreml_tensor) => { /* usar CoreML */ },
    //     Err(_) => { /* respaldo a Metal/CPU */ }
    // }
    
    let use_coreml = false; // Simular fallo CoreML
    
    if use_coreml {
        println!("✅ Operación CoreML exitosa");
    } else {
        println!("⚠️ CoreML no disponible, usando respaldo");
        
        // Respaldo a Metal GPU
        let start = Instant::now();
        let result = complex_tensor.matmul(&complex_tensor);
        let fallback_time = start.elapsed();
        
        println!("✅ Operación de respaldo completada");
        println!("⏱️ Tiempo de respaldo: {:?}", fallback_time);
        println!("📐 Forma del resultado: {:?}", result.shape());
    }
    
    println!("\n📝 Estrategia de respaldo:");
    println!("  1. Intentar CoreML (mejor rendimiento)");
    println!("  2. Respaldo a Metal GPU (buena compatibilidad)");
    println!("  3. Respaldo final a CPU (máxima compatibilidad)");
}

demonstrate_fallback_behavior();

## Resumen y Próximos Pasos

In [ ]:
println!("📋 Resumen de Integración CoreML de RusTorch (Rust Kernel):");
println!();
println!("✅ Características demostradas:");
println!("  • Verificación de disponibilidad CoreML");
println!("  • Creación y gestión de dispositivos");
println!("  • Configuración del backend");
println!("  • Operaciones básicas de tensores");
println!("  • Evaluación comparativa de rendimiento");
println!("  • Selección inteligente de dispositivo");
println!("  • Comportamiento de respaldo");
println!();
println!("🚧 Área de desarrollo:");
println!("  • Implementación completa de operaciones CoreML");
println!("  • Optimización de transferencia de memoria");
println!("  • Soporte ampliado para tipos de tensor");
println!("  • Perfilado detallado de rendimiento");
println!("  • Integración con pipelines de ML");
println!();
println!("🎯 Próximos pasos recomendados:");
println!("  1. Probar con modelos CoreML preentrenados");
println!("  2. Evaluar comparativamente con otros backends");
println!("  3. Optimizar para casos de uso específicos");
println!("  4. Implementar en aplicaciones de producción");
println!();

#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    if rustorch::backends::DeviceManager::is_coreml_available() {
        println!("🎉 ¡Todas las características CoreML están disponibles para pruebas!");
    } else {
        println!("⚠️ CoreML está habilitado pero no disponible en este sistema");
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ Construya con características CoreML para funcionalidad completa");
}

println!("\n🚀 Listo para el desarrollo avanzado de CoreML con RusTorch!");